In [1]:
#import os
#os.environ["OMP_NUM_THREADS"]='32'
import numpy as np
from scipy import linalg
from functools import reduce
np.set_printoptions(precision=2)
import matplotlib.pyplot as plt
import tensornetwork as tn
import time,math,gc,itertools
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigsh
from scipy.sparse.linalg import svds
from tqdm import tqdm
import opt_einsum
import sys
from colorsys import hls_to_rgb
def show(z):
    if len(z.shape)==1:
        z=z.reshape((1,z.shape[0]))
    r,arg = np.abs(z),np.angle(z)
    h,s = (arg + np.pi)  / (2 * np.pi) + 0.5,0.8
    l=r**(1/2.2) #gamma correction
    l=l/(2*np.max(l))
    c = np.array(np.vectorize(hls_to_rgb) (h,l,s)).swapaxes(0,2).swapaxes(0,1)
    plt.imshow(c)
    plt.show()
    
tn.set_default_backend("numpy")

In [2]:
import threading,time,datetime
import io,sys
from tqdm import tqdm
from IPython.display import clear_output

class BackgroundRunning:
    def __init__(self, job):
        self.result=None
        self.display=io.StringIO()
        self.job=job
    def start(self):
        self.start_time=time.time()
        def run_thread():
            self.result=job(self.display)
        self.thread=threading.Thread(target=run_thread,args=())
        self.thread.start()
                
    def showStatus(self):
        def sec2str(sec):
            return str(datetime.timedelta(seconds=int(sec)))
        
        clear_output(wait=True)
        elapsed_time=time.time()-self.start_time
        s='[{}]'.format(sec2str(elapsed_time))
        print(s)
        s=self.display.getvalue()
        x=s.rfind('\r')
        if x!=-1:
            s=s[x+1:]
        print(s)
        if self.is_done():
            print('done')
    def monitor(self,duration=10):
        for i in range(duration*4):
            self.showStatus()
            if self.is_done():
                return
            time.sleep(.25)
        print('to be continue...')
    def is_done(self):
        return not self.thread.is_alive()

In [3]:
def getCG(n):
    """shape:[n+1]+[2]*n
    """
    if n==0:
        return np.eye(1)
    CG=np.zeros((n+1,)+(2,)*n)
    for i in range(2**n):
        indices=tuple(map(int,bin(i)[2:].zfill(n)))
        m=np.sum(indices)
        CG[(m,)+indices]=1
    CG=np.array(list(map(lambda x:x/linalg.norm(x),CG.reshape(n+1,-1)))).reshape(CG.shape)
    return CG

assert getCG(3).shape==(4,2,2,2)
CGs=[getCG(i) for i in range(7)]
singlet = np.array([[0,1],[-1,0]])
assert (singlet.dot(singlet)==-np.eye(2)).all()
assert (singlet==-singlet.T).all()

In [4]:
def contract_physical_edges(physical_edges1,physical_edges2):
    """
    Parameters
    ----------
    edges1,edges2:map vertice_id->edge
    """
    for k in physical_edges1:
        if k in physical_edges2:
            physical_edges1[k]^physical_edges2[k]
def contract_disconnected_tensor_network(nodes,edges):
    newNodes=[]
    nodes=set(nodes)
    while len(nodes)>0:
        newSet=set(tn.reachable(next(iter(nodes))))
        newEdges=list(filter(lambda x:x.get_nodes()[0] in newSet or x.get_nodes()[1] in newSet,edges))
        #print(newSet,[[x.get_nodes()[0],x.get_nodes()[1]] for x in newEdges])
        newNode=tn.contractors.auto(newSet,newEdges)
        nodes=nodes.difference(newSet)
        newNodes.append(newNode)
    result=tn.outer_product_final_nodes(newNodes,edges)
    return result 

In [5]:
assert sys.version_info>=(3,7) #will use dict ordering feature
class AKLT_Tensor:
    """
    Attributes
    ----------
    physical_edges:sorted dict vertice_id->edge
    virtual_edges:sorted dict vertice_id->edge
    nodes:sorted dict vertice_id->node or None after SVD
    physical_dimension:int
    virtual_dimension:int
    
    """
    def __init__(self,spins,conns,div):
        """
        Parameters
        ----------
        spins:[2,2,2,2]
            the spins for the vertices of the parent graph
        conns:[[0,1],[1,2],[2,3]]
            array of pair of vertice_ids for edges in the parent graph
        div:[1,2]
            the set of vertice_ids for vertices in the subgraph
        """        
        self.nodes=dict()
        self.physical_edges=dict()
        self.virtual_edges=dict()
        
        curIndice=[1]*len(spins)
        for vid in div:
            self.nodes[vid]=tn.Node(CGs[spins[vid]])# shape: [n+1]+[2]*n
        for vid1,vid2 in conns:
            if vid1 in div and vid2 in div:
                n1=self.nodes[vid1]
                n2=tn.Node(singlet)
                n3=self.nodes[vid2]

                edges=n1.get_all_edges()
                edges[curIndice[vid1]]=n2[1]
                n1=tn.contract(n1[curIndice[vid1]]^n2[0]).reorder_edges(edges)
                self.nodes[vid1]=n1

                n1[curIndice[vid1]]^n3[curIndice[vid2]]
                curIndice[vid1]+=1
                curIndice[vid2]+=1
        for vid in div:
            n1=self.nodes[vid]
            self.physical_edges[vid]=n1[0]
            num=spins[vid]+1-curIndice[vid]
            if num==0:
                pass
            elif num==1:
                self.virtual_edges[vid]=n1[curIndice[vid]]
            else:
                n2=tn.Node(CGs[num].conj())
                edges=n1.get_all_edges()[0:curIndice[vid]]+[n2[0]]
                for i in range(num):
                    n1[curIndice[vid]+i]^n2[1+i]
                n1=tn.contract_between(n1,n2,output_edge_order=edges)
                self.nodes[vid]=n1
                self.virtual_edges[vid]=edges[-1]
        
        self.nodes=dict(sorted(self.nodes.items()))
        self.physical_edges=dict(sorted(self.physical_edges.items()))
        self.virtual_edges=dict(sorted(self.virtual_edges.items()))
        
        self.physical_dimensions=[e.dimension for e in self.physical_edges.values()]
        self.virtual_dimensions=[e.dimension for e in self.virtual_edges.values()]
        self.physical_dimension=np.prod(self.physical_dimensions)
        self.virtual_dimension=np.prod(self.virtual_dimensions)
        

    
    def print_dimensions(self):
        print("node shapes:",[(i,n.shape) for i,n in self.nodes.items()])
        print("physical dimensions:",self.physical_dimensions)
        print("virtual dimensions:",self.virtual_dimensions)
        print("contracted size:",self.physical_dimension,"x",self.virtual_dimension,"=",self.physical_dimension*self.virtual_dimension)
    
    def print_topology(self):
        inm={n:i for i,n in self.nodes.items()}
        toPrint=[]
        for n in self.nodes.values():
            for e in n.edges:
                if e in self.physical_edges.values():
                    assert e.node2 is None
                    toPrint.append([inm[e.node1],'p',e.dimension])
                elif e in self.virtual_edges.values():
                    assert e.node2 is None
                    toPrint.append([inm[e.node1],'v',e.dimension])
                else:
                    toPrint.append([e.node1 and inm[e.node1],e.node2 and inm[e.node2],e.dimension])
        print(toPrint)
    def copy(self):
        node_dict,edge_dict=tn.copy(self.nodes.values())
        rtval=object.__new__(self.__class__)
        
        rtval.nodes={k:node_dict[v] for k,v in self.nodes.items()}
        rtval.physical_edges={k:edge_dict[v] for k,v in self.physical_edges.items()}
        rtval.virtual_edges={k:edge_dict[v] for k,v in self.virtual_edges.items()}
        rtval.physical_dimensions=self.physical_dimensions.copy()
        rtval.virtual_dimensions=self.virtual_dimensions.copy()
        rtval.physical_dimension=self.physical_dimension
        rtval.virtual_dimension=self.virtual_dimension
        
        return rtval
    def get_tensor(self):
        t=self.copy()
        return contract_disconnected_tensor_network(
            list(t.nodes.values()),
            list(t.physical_edges.values())+list(t.virtual_edges.values())
        ).tensor


In [6]:
class AKLT_Physical_Basis:
    def __init__(self,spins,conns,div,validate=True):
        """
        Parameters
        ----------
        spins:[2,2,2,2]
            the spins for the vertices of the parent graph
        conns:[[0,1],[1,2],[2,3]]
            array of pair of vertice_ids for edges in the parent graph
        div:[1,2]
            the set of vertice_ids for vertices in the subgraph
        """
        tensor=AKLT_Tensor(spins,conns,div)
        # assert tn.check_correct(tensor.nodes.values())==None wrong for non-connected graph

        
        t1,t2=tensor.copy(),tensor.copy()
        contract_physical_edges(t1.physical_edges,t2.physical_edges)
        print("begin contract");start_time=time.time()
        mm=contract_disconnected_tensor_network(
            list(t1.nodes.values())+list(t2.nodes.values()),
            list(t1.virtual_edges.values())+list(t2.virtual_edges.values())
        ).tensor
        print('time used: ',time.time()-start_time)
        
        print("begin SVD for ",tensor.virtual_dimension," x ",tensor.virtual_dimension);start_time=time.time()
        s2,v=linalg.eigh(mm.reshape((tensor.virtual_dimension,tensor.virtual_dimension)))
        print('time used: ',time.time()-start_time)
        s=np.sqrt(s2)
        print("maxmin singular values: ",max(s),min(s)," total ",len(s))
        vh=v.conj().T
        #assert np.allclose(v@vh,np.eye(v.shape[0]))
        #assert np.allclose(vh@v,np.eye(vh.shape[0]))
        
        #new_tensor=v@np.diag(1/s)
        new_tensor=v@np.diag(1/s)@vh
        new_node=tn.Node(new_tensor.reshape(tensor.virtual_dimensions+[new_tensor.shape[1]]))
        
        for i,e in enumerate(tensor.virtual_edges.values()):
            new_node[i]^e
                
        self.all_nodes=[new_node]+list(tensor.nodes.values())
        self.physical_edges=tensor.physical_edges
        self.virtual_edge=new_node[-1]
        self.virtual_dimension=new_tensor.shape[1]
        self.physical_dimensions=tensor.physical_dimensions.copy()
        self.physical_dimension=tensor.physical_dimension
        self.singular_values=s
        self.v_tensor=v
        
        if validate:
            print('start validate');start_time=time.time()
            self.test_unitary()
            self.test_idempotent()
            print('validate passed')
            print('time used: ',time.time()-start_time)
        
    def print_dimensions(self):
        print("node shapes:",[n.shape for n in self.all_nodes])
        print("physical dimensions:",self.physical_dimensions)
        print("virtual dimension:",self.virtual_dimension)
        print("contracted size:",self.physical_dimension,"x",self.virtual_dimension,"=",self.physical_dimension*self.virtual_dimension)
    
    def copy(self):
        node_dict,edge_dict=tn.copy(self.all_nodes)
        rtval=object.__new__(self.__class__)
        
        rtval.all_nodes=[node_dict[v] for v in self.all_nodes]
        rtval.physical_edges={k:edge_dict[v] for k,v in self.physical_edges.items()}
        rtval.virtual_dimension=self.virtual_dimension
        rtval.virtual_edge=edge_dict[self.virtual_edge]
        rtval.physical_dimensions=self.physical_dimensions.copy()
        rtval.physical_dimension=self.physical_dimension
        rtval.singular_values=self.singular_values.copy()
        rtval.v_tensor=self.v_tensor.copy()
        
        return rtval
    def get_tensor(self):
        t=self.copy()
        return contract_disconnected_tensor_network(
            t.all_nodes,
            list(t.physical_edges.values())+[t.virtual_edge]
        ).tensor
    
    def test_unitary(self):
        a,b=self.copy(),self.copy()
        contract_physical_edges(a.physical_edges,b.physical_edges)
        t=contract_disconnected_tensor_network(
            a.all_nodes+b.all_nodes,
            [a.virtual_edge,b.virtual_edge]
        ).tensor
        assert np.allclose(t,np.eye(self.virtual_dimension))
    def test_idempotent(self):
        def E(v):
            n0=tn.Node(v.reshape(self.physical_dimensions))
            n0_physical_edges={k:n0[i] for i,k in enumerate(self.physical_edges)}
            a,b=self.copy(),self.copy()
            contract_physical_edges(n0_physical_edges,a.physical_edges)
            a.virtual_edge^b.virtual_edge
            nodes=a.all_nodes+b.all_nodes+[n0]
            edges=list(b.physical_edges.values())
            n1=contract_disconnected_tensor_network(nodes,edges)
            return n1.tensor.flatten()
        v=np.random.random(self.physical_dimension)
        assert np.allclose(E(v),E(E(v)))
        
a=AKLT_Physical_Basis([4,2],[[0,1]],[0,1])
b=AKLT_Tensor([4,2],[[0,1]],[0,1])
pd=a.physical_dimension
vd=a.virtual_dimension
ua=a.get_tensor().reshape(pd,-1)@a.v_tensor
sa=np.diag(a.singular_values)
va=a.v_tensor
tb=b.get_tensor().reshape(pd,-1)
assert np.allclose(ua@sa@va.T,tb)
assert np.allclose(va.T@va,np.eye(vd))
assert np.allclose(va@va.T,np.eye(vd))
assert np.allclose(ua.T@ua,np.eye(vd))
assert np.allclose(ua@ua.T,ua@ua.T@ua@ua.T)

a.test_unitary()
a.test_idempotent()


begin contract
time used:  0.0015180110931396484
begin SVD for  8  x  8
time used:  0.03265690803527832
maxmin singular values:  1.1180339887498951 0.8660254037844386  total  8
start validate
validate passed
time used:  0.018136262893676758


In [13]:
# diamond8 n=1 type8 x12
# <0.06 0.01tol 2.1199999999999948 < 2.12 == True
# <0.03 0.01tol 2.0700792853589918 < 2.06 == False
# -0.04050477911678796 -0.9894952208832121
# <0.03 0tol 2.0708429296843014 < 2.06 == False 1121it [17:45:08, 57.01s/it]
# -0.0413101538815735 -0.9886898461184266
# svd method: 0.04131015388150461 811it [8:45:54, 38.91s/it]

#new
# 21it [06:30, 18.60s/it] 2.2000000000000006 < 2.2 == False  -0.1000000000000007 -0.9999999999999993 eta0  0.1 tol  0.01 eta  0.1000000000000007
# 981it [4:54:43, 18.03s/it] 2.07084292968422 < 2.06 == False eta0  0.03 tol  0 eta  0.04131015388148773

spins=[2,2,2,4,2, 2,2,2,4,2, 2,2,2,4,2, 4, 2,2,2,4,2, 2,2,2,4,2, 2,2,2,4,2, 2,2,2,4,2, 4]
conns=[1,4,2,4,3,4,4,5,5,16, 6,9,7,9,8,9,9,10,10,16, 11,14,12,14,13,14,14,15,15,16, 16,17, 17,20,18,20,19,20,21,20, 22,25,23,25,24,25,25,26,26,37, 27,30,28,30,29,30,30,31,31,37, 32,35,33,35,34,35,35,36,36,37, 37,21]
connsL=np.array(conns).reshape(-1,2)-1
connsM=connsL
connsR=connsL
connsLM=connsL
connsMR=connsL
left=range(0,16)
middle=range(16,21)
right=range(21,37)

In [10]:
# kagome
# 21it [00:18,  1.13it/s] eta0  0.1 tol  0.01 eta  0.16673098088249844
# 1151it [16:41,  1.15it/s] eta0  0.1 tol  0 eta  0.17067852082351342

#spins=[4,4,4,4,4,4, 4,4,4, 4,4,4,4,4,4]
#def tri(a,b,c):
#    return [a,b,b,c,c,a]
#conns=tri(1,2,3)+tri(3,4,8)+tri(4,5,6)+tri(7,8,9)+tri(9,11,13)+tri(10,11,12)+tri(13,14,15)
#connsL=np.array(conns).reshape(-1,2)-1
#connsM=connsL
#connsR=connsL
#connsLM=connsL
#connsMR=connsL
#left=range(0,6)
#middle=range(6,9)
#right=range(9,15)

In [9]:
# kagome bigger 1
# spins=[4]*27
# def tri(a,b,c):
#     return [a,b,b,c,c,a]
# conns=tri(1,2,3)+tri(4,5,6)+tri(7,8,9)+tri(10,11,12)+tri(13,14,15)+tri(16,17,18)+tri(19,20,21)+tri(22,23,24)\
#     +tri(25,26,27)+tri(3,4,8)+tri(9,10,14)+tri(15,16,20)+tri(21,22,26)
# connsL=np.array(conns).reshape(-1,2)-1
# connsM=connsL
# connsR=connsL
# connsLM=connsL
# connsMR=connsL
# left=range(0,12)
# middle=range(12,15)
# right=range(15,27)

In [10]:
# kagome bigger mod 1
# spins=[4]*27
# def tri(a,b,c):
#     return [a,b,b,c,c,a]
# conns=tri(1,2,3)+tri(4,5,6)+tri(7,8,9)+tri(10,11,12)+tri(13,14,15)+tri(16,17,18)+tri(19,20,21)+tri(22,23,24)\
#     +tri(25,26,27)+tri(3,4,8)+tri(9,10,14)+tri(15,16,20)+tri(21,22,26)\
#     +[6,11,18,23]
# connsL=np.array(conns).reshape(-1,2)-1
# connsM=connsL
# connsR=connsL
# connsLM=connsL
# connsMR=connsL
# left=range(0,12)
# middle=range(12,15)
# right=range(15,27)

In [11]:
# square HH diag
spins=[4]*28
def quad(a,b,c,d):
    return [a,b,a,c,b,d,c,d]
conns=quad(1,2,3,4)+quad(5,6,7,8)+quad(9,10,11,12)+quad(13,14,15,16)+quad(17,18,19,20)+quad(21,22,23,24)+quad(25,26,27,28)\
        +quad(4,7,10,13)+quad(16,19,22,25)\
        +[3,9,8,14,15,21,20,26]
connsL=np.array(conns).reshape(-1,2)-1
connsM=connsL
connsR=connsL
connsLM=connsL
connsMR=connsL
left=range(0,12)
middle=range(12,16)
right=range(16,28)

In [8]:
# square HH side
spins=[4]*24
def quad(a,b,c,d):
    return [a,b,a,c,b,d,c,d]
connsLM0=quad(1,2,3,4)+quad(5,6,7,8)+quad(9,10,11,12)+quad(13,14,15,16)+quad(4,11,6,13)+[3,5,12,14]
connsMR0=quad(9,10,11,12)+quad(13,14,15,16)+quad(17,18,19,20)+quad(21,22,23,24)+quad(12,19,14,21)+[10,17,16,23]
connsL=np.array(connsLM0).reshape(-1,2)-1
connsM=np.array(connsMR0).reshape(-1,2)-1
connsR=np.array(connsMR0).reshape(-1,2)-1
connsLM=np.array(connsLM0).reshape(-1,2)-1
connsMR=np.array(connsMR0).reshape(-1,2)-1
left=range(0,8)
middle=range(8,16)
right=range(16,24)

In [14]:
memory_limit=100*1024*1024*1024
left,middle,right=set(left),set(middle),set(right)
assert all(all(i<j for i in left)for j in middle)
assert all(all(i<j for i in middle)for j in right)
assert len(left)+len(middle)+len(right)==max(right)+1

UL=AKLT_Physical_Basis(spins,connsL,left,validate=False)
UM=AKLT_Physical_Basis(spins,connsM,middle,validate=False)
UR=AKLT_Physical_Basis(spins,connsR,right,validate=False)
ULM=AKLT_Physical_Basis(spins,connsLM,left.union(middle),validate=False)
UMR=AKLT_Physical_Basis(spins,connsMR,middle.union(right),validate=False)

dimL,dimM,dimR=UL.virtual_dimension,UM.virtual_dimension,UR.virtual_dimension
dimLMR=dimL*dimM*dimR


def E(v):
    n0=tn.Node(v.reshape((UL.virtual_dimension,UM.virtual_dimension,UR.virtual_dimension)))
    n1,n2=UL.copy(),UM.copy()
    n3=ULM.copy()
    n4=ULM.copy()
    n5,n6=UL.copy(),UM.copy()
    n0[0]^n1.virtual_edge
    n0[1]^n2.virtual_edge
    contract_physical_edges(n1.physical_edges,n3.physical_edges)
    contract_physical_edges(n2.physical_edges,n3.physical_edges)
    n3.virtual_edge^n4.virtual_edge
    contract_physical_edges(n5.physical_edges,n4.physical_edges)
    contract_physical_edges(n6.physical_edges,n4.physical_edges)
    
    nodes=[n0]+[n for N in [n1,n2,n3,n4,n5,n6] for n in N.all_nodes]
    edges=[n5.virtual_edge,n6.virtual_edge,n0[2]]
    
    n7=tn.contractors.auto(nodes,edges,memory_limit=memory_limit)
    rtval=n7.tensor.reshape(-1)
    return rtval

def F(v):
    n0=tn.Node(v.reshape((UL.virtual_dimension,UM.virtual_dimension,UR.virtual_dimension)))
    n1,n2=UM.copy(),UR.copy()
    n3=UMR.copy()
    n4=UMR.copy()
    n5,n6=UM.copy(),UR.copy()
    n0[1]^n1.virtual_edge
    n0[2]^n2.virtual_edge
    contract_physical_edges(n1.physical_edges,n3.physical_edges)
    contract_physical_edges(n2.physical_edges,n3.physical_edges)
    n3.virtual_edge^n4.virtual_edge
    contract_physical_edges(n5.physical_edges,n4.physical_edges)
    contract_physical_edges(n6.physical_edges,n4.physical_edges)
    
    nodes=[n0]+[n for N in [n1,n2,n3,n4,n5,n6] for n in N.all_nodes]
    edges=[n0[0],n5.virtual_edge,n6.virtual_edge]
    
    n7=tn.contractors.auto(nodes,edges,memory_limit=memory_limit)
    rtval=n7.tensor.reshape(-1)
    return rtval
    
print('dimL*dimM*dimR',dimL*dimM*dimR)
print("begin apply EF");start_time = time.time()
v=np.random.random(dimLMR)
Ev=E(v)
Fv=F(v)
assert np.allclose(E(Ev),Ev)
assert np.allclose(F(Fv),Fv)
del v
print("time used: ",time.time() - start_time)
print('done')

begin contract
time used:  0.028612136840820312
begin SVD for  1024  x  1024
time used:  0.4933764934539795
maxmin singular values:  3.926909047381791 2.5892585428639543  total  1024
begin contract
time used:  0.00789022445678711
begin SVD for  16  x  16
time used:  0.0016243457794189453
maxmin singular values:  1.3919410907075056 1.1180339887498951  total  16
begin contract
time used:  0.02190423011779785
begin SVD for  1024  x  1024
time used:  0.45847368240356445
maxmin singular values:  3.9269090473817907 2.5892585428639543  total  1024
begin contract
time used:  0.10562896728515625
begin SVD for  4096  x  4096
time used:  8.482908248901367
maxmin singular values:  6.981135754346303 4.4158460493159835  total  4096
begin contract
time used:  0.11421465873718262
begin SVD for  4096  x  4096
time used:  7.487294912338257
maxmin singular values:  6.981135754346305 4.415846049315985  total  4096
dimL*dimM*dimR 16777216
begin apply EF
time used:  11.082455158233643
done


In [ ]:
import psutil,os
print('memory usage',psutil.Process(os.getpid()).memory_info().rss)

1-z eta>0 => gapped

eta=sup(noninteger a)

eta0= estimated upper bound of eta

if result is near to 2+2 eta0 then reduce eta

if result is larger than 2+2 eta0 then for checking double root, increase eta


In [41]:
# 1-z eta>0 => gapped
# eta=sup(noninteger a)
# eta0= estimated upper bound of eta
# if result is near to 2+2 eta0 then reduce eta
# if result is larger than 2+2 eta0 then for checking double root, increase eta


eta0=0.1
tol=0.01
threshold=2*(1+eta0)




def job(file):
    pbar=tqdm(file=file)
    oldeigs=[0]
    def OP(v):
        Ev=E(v)
        gc.collect(2)
        Fv=F(v)
        gc.collect(2)
        EFv=E(Fv)
        gc.collect(2)
        FEv=F(Ev)
        gc.collect(2)
        rtval=(2+eta0)*(Ev+Fv)-(EFv+FEv)
        pbar.update(1)
        del Ev,Fv,EFv,FEv
        gc.collect(2)
        eig=np.linalg.norm(rtval)/np.linalg.norm(v)
        pbar.set_postfix_str(str(oldeigs[-1])+'>'+str(eig))
        oldeigs.append(eig)
        return rtval
    op=LinearOperator(shape=(dimLMR,dimLMR),matvec=OP)
    eigenvalues, eigenvectors = eigsh(op, k=1,tol=tol)
    pbar.close()
    return eigenvalues,eigenvectors

bg=BackgroundRunning(job)
bg.start()



/tmp/ipykernel_499369/1811551087.py:33: RuntimeWarning: invalid value encountered in double_scalars
  eig=np.linalg.norm(rtval)/np.linalg.norm(v)


In [46]:


bg.monitor(100)
if(bg.is_done()):
    eigenvalues, eigenvectors=bg.result

    e=eigenvalues[0]
    if e<=threshold:
        print(e,'<=',threshold,'try increase eta0')
    else:
        print(e,'>',threshold,'extract a. please check which root is correct by increase eta0 (but eta0 < smaller a)')
        print("possible a:")
        print(-0.5*(-1-eta0+ np.sqrt(9 - 4 *e + 6 *eta0 + eta0**2)))
        print(-0.5*(-1-eta0- np.sqrt(9 - 4 *e + 6 *eta0 + eta0**2)))

    smallRoot=-0.5*(-1-eta0+ np.sqrt(9 - 4 *e + 6 *eta0 + eta0**2))
    print("eta0 ",eta0,"tol ",tol,"eta ",smallRoot)

[0:04:36]
21it [00:26,  1.28s/it, 2.078389213495868>2.086729340219865]

done
2.2556993722788063 > 2.2 extract a. please check which root is correct by increase eta0 (but eta0 < smaller a)
possible a:
0.16685429961802567
0.9331457003819744
eta0  0.1 tol  0.01 eta  0.16685429961802567


In [1]:
1+1

2